In [1]:
import sys
print(sys.executable)


c:\Users\Windows 11\Desktop\AI_Engineer\Projet_2_Requetez_des_services_IA\.venv\Scripts\python.exe


## 1. Configuration Initiale et Importations


In [ ]:
import os

import requests

#from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import tqdm
from tqdm import tqdm 
from tqdm.notebook import tqdm
import base64
import io

ModuleNotFoundError: No module named 'matplotlib'

In [2]:

from dotenv import load_dotenv
load_dotenv(dotenv_path=r"C:\Users\Windows 11\Desktop\AI_Engineer\Projet_2_Requetez_des_services_IA\.venv\.env")
api_key = os.getenv("API_KEY")
# print les 5 dernier digit de la clé
if api_key:
    api_token = api_key
    print(f"API Key chargée : {api_key[:5]}***")  # masque pour affichage
else:
    api_token = None
    print("API Key non trouvée. Vérifiez le fichier .env et la variable 'api_token'.")



NameError: name 'os' is not defined

## 2. API d'Inférence Hugging Face


In [ ]:
from dotenv import load_dotenv

load_dotenv()  # .env automatiquement trouvé à la racine du projet

api_key = os.getenv("API_KEY")
if api_key:
    api_token = api_key
    print(f"API Key chargée : {api_key[:5]}***")  # masque pour affichage des 5 premières caractères)
else:
    api_token = None
    print("API Key non trouvée. Vérifiez le fichier .env et la variable 'api_token'.")



ModuleNotFoundError: No module named 'dotenv'

test site

## 3. Fonctions Utilitaires pour le Traitement des Masques


In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================

import os
import time
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import base64
import io


# ============================================================================
# MAPPAGE DES CLASSES ET COULEURS
# ============================================================================

# Mappage classe anglaise -> ID numérique (tel que retourné par l'API)
CLASS_MAPPING = {
    "Background": 0,
    "Hat": 1,
    "Hair": 2,
    "Sunglasses": 3,
    "Upper-clothes": 4,
    "Skirt": 5,
    "Pants": 6,
    "Dress": 7,
    "Belt": 8,
    "Left-shoe": 9,
    "Right-shoe": 10,
    "Face": 11,
    "Left-leg": 12,
    "Right-leg": 13,
    "Left-arm": 14,
    "Right-arm": 15,
    "Bag": 16,
    "Scarf": 17
}

# Mappage anglais -> français pour l'affichage de la légende
CLASS_NAMES_FRENCH = {
    "Background": "Arrière-plan",
    "Hat": "Chapeau",
    "Hair": "Cheveux",
    "Sunglasses": "Lunettes de soleil",
    "Upper-clothes": "Haut (vêtement)",
    "Skirt": "Jupe",
    "Pants": "Pantalon",
    "Dress": "Robe",
    "Belt": "Ceinture",
    "Left-shoe": "Chaussure gauche",
    "Right-shoe": "Chaussure droite",
    "Face": "Visage",
    "Left-leg": "Jambe gauche",
    "Right-leg": "Jambe droite",
    "Left-arm": "Bras gauche",
    "Right-arm": "Bras droit",
    "Bag": "Sac",
    "Scarf": "Écharpe"
}

COLOR_MAPPING = {
    0: (0, 0, 0),
    1: (255, 255, 0),
    2: (255, 165, 0),
    3: (255, 0, 255),
    4: (255, 0, 0),
    5: (255, 255, 0),
    6: (0, 128, 0),
    7: (0, 0, 255),
    8: (128, 0, 128),
    9: (255, 255, 0),
    10: (0, 0, 255),
    11: (135, 206, 235),
    12: (173, 216, 230),
    13: (173, 216, 230),
    14: (173, 216, 230),
    15: (173, 216, 230),
    16: (255, 165, 0),
    17: (128, 0, 128)
}

# ============================================================================
# FONCTIONS UTILITAIRES
# ============================================================================

def get_image_dimensions(img_path):
    """Récupère les dimensions (largeur, hauteur) d'une image."""
    original_image = Image.open(img_path)
    return original_image.size


def decode_base64_mask(base64_string, width, height):
    """Décode un masque encodé en base64 en tableau NumPy."""
    mask_data = base64.b64decode(base64_string)
    mask_image = Image.open(io.BytesIO(mask_data))
    mask_array = np.array(mask_image)
    if len(mask_array.shape) == 3:
        mask_array = mask_array[:, :, 0]
    mask_image = Image.fromarray(mask_array).resize((width, height), Image.NEAREST)
    return np.array(mask_image)


def create_masks(results, width, height):
    """
    Combine les masques de plusieurs classes en un seul masque.
    
    IMPORTANT: L'ordre d'application est crucial:
    - Initialiser avec Background (0)
    - Appliquer NON-background en premier
    - Appliquer Background en dernier
    """
    combined_mask = np.zeros((height, width), dtype=np.uint8)

    # Afficher les classes détectées pour déboguer
    print(f"    Classes détectées: {[r['label'] for r in results]}")

    # Traiter les masques non-Background en premier
    for result in results:
        label = result['label']
        class_id = CLASS_MAPPING.get(label, 0)
        if class_id == 0:
            continue
        
        mask_array = decode_base64_mask(result['mask'], width, height)
        pixels_assigned = np.count_nonzero(mask_array)
        print(f"    {label} (ID={class_id}): {pixels_assigned} pixels")
        combined_mask[mask_array > 0] = class_id

    # Traiter Background en dernier
    for result in results:
        if result['label'] == 'Background':
            mask_array = decode_base64_mask(result['mask'], width, height)
            pixels_assigned = np.count_nonzero(mask_array)
            print(f"    Background (ID=0): {pixels_assigned} pixels")
            # Ne pas écraser les classes déjà assignées
            combined_mask[(combined_mask == 0) & (mask_array > 0)] = 0

    return combined_mask


def colorize_segmentation_mask(mask_array):
    """Convertit un masque en niveaux de gris en image RGB colorisée."""
    height, width = mask_array.shape
    colored_mask = np.zeros((height, width, 3), dtype=np.uint8)
    
    for class_id, color_rgb in COLOR_MAPPING.items():
        class_mask = (mask_array == class_id)
        colored_mask[class_mask] = color_rgb

    return colored_mask


def create_overlay(original_img, colored_mask, alpha=0.6):
    """
    Crée une superposition du masque colorisé sur l'image originale.
    
    Args:
        original_img: Image PIL originale
        colored_mask: Array NumPy RGB du masque colorisé
        alpha: Transparence du masque (0-1)
    
    Returns:
        Image PIL de la superposition
    """
    # Redimensionner original_img si nécessaire
    if original_img.size != (colored_mask.shape[1], colored_mask.shape[0]):
        original_img = original_img.resize(
            (colored_mask.shape[1], colored_mask.shape[0]),
            Image.Resampling.LANCZOS
        )
    
    # Convertir original_img en array
    original_array = np.array(original_img, dtype=np.float32)
    colored_mask_float = colored_mask.astype(np.float32)
    
    # Appliquer l'overlay avec alpha blending
    overlay = (1 - alpha) * original_array + alpha * colored_mask_float
    overlay = overlay.astype(np.uint8)
    
    return Image.fromarray(overlay)


## 4. Segmentation d'une Seule Image

In [ ]:


headers = {
    "Authorization": f"Bearer {api_token}"
}
 

if image_paths:
    single_image_path = image_paths[0] # Prenons la première image de notre liste
    print(f"Traitement de l'image : {single_image_path}")

    try:
        # Lire l'image en binaire
        with open(single_image_path, "rb") as f:
            image_data = f.read()
       
        

        # Maintenant, utilisé l'API huggingface
        response = requests.post(
            API_URL, 
            headers={**headers, "Content-Type": "image/png"}, 
            data=image_data)
        response.raise_for_status()  # Vérifie si la requête a réussi
        # ainsi que les fonctions données plus haut pour ségmenter vos images.
        results = response.json()
        print("Réponse brute de l'API :")
        print(results)

        width, height = get_image_dimensions(single_image_path)
        masks = create_masks(results, width, height)

    except Exception as e:
        print(f"Une erreur est survenue : {e}")
else:
    print("Aucune image à traiter. Vérifiez la configuration de 'image_dir' et 'max_images'.")


## 5. Segmentation de Plusieurs Images (Batch)


In [ ]:
def segment_images_batch(list_of_image_paths, max_api_calls=50, delay_between_calls=1):
    """
    Segmente une liste d'images en utilisant l'API Hugging Face avec gestion robuste.
    
    Cette fonction traite les images de manière séquentielle en appelant l'API
    d'inférence Hugging Face. Elle implémente:
    - Une limite de requêtes API pour éviter les surcharges (max 50 par défaut)
    - Un délai entre les appels pour respecter les rate limits de l'API
    - Une gestion d'exceptions exhaustive pour chaque type d'erreur
    - Un logging détaillé de chaque étape du traitement
    
    Args:
        list_of_image_paths (list): Liste des chemins vers les images PNG à segmenter.
        max_api_calls (int): Nombre maximum d'appels API autorisés (défaut: 50).
                            Les images au-delà de cette limite seront ignorées.
        delay_between_calls (int): Délai en secondes entre chaque appel API (défaut: 1).
                                  Augmenter cette valeur si vous rencontrez des
                                  erreurs 429 (Too Many Requests).
    
    Returns:
        list: Liste de masques de segmentation (numpy arrays uint8) correspondant
              aux images traitées. Contient None pour les images non traitées
              ou en cas d'erreur. La liste a la même longueur que list_of_image_paths.
    
    Levées:
        Aucune exception n'est levée. Les erreurs sont capturées et loggées.
        Chaque image problématique est enregistrée comme None dans la liste retournée.
    
    Exemple:
        >>> chemins = ['img1.png', 'img2.png', 'img3.png']
        >>> resultats = segment_images_batch(chemins, max_api_calls=50, delay_between_calls=1)
        >>> print(len(resultats))  # 3
    """
    batch_seg_results = []
    total_images = len(list_of_image_paths)
    api_calls_made = 0
    images_skipped = 0

    print(f"Limite d'appels API: {max_api_calls}")
    print(f"Délai entre appels: {delay_between_calls}s\n")

    for idx, img_path in enumerate(list_of_image_paths, 1):
        
        # Vérifier la limite d'appels API
        if api_calls_made >= max_api_calls:
            print(f"[{idx}/{total_images}] LIMITE ATTEINTE - Image ignorée")
            batch_seg_results.append(None)
            images_skipped += 1
            continue
        
        print(f"[{idx}/{total_images}] {os.path.basename(img_path)}")
        
        try:
            # Étape 1: Lire le fichier image
            if not os.path.exists(img_path):
                raise FileNotFoundError(f"Fichier introuvable: {img_path}")
            
            with open(img_path, "rb") as f:
                image_data = f.read()
            
            # Étape 2: Envoyer requête POST à l'API HF
            try:
                response = requests.post(
                    API_URL,
                    headers={**headers, "Content-Type": "image/png"},
                    data=image_data,
                    timeout=30
                )
                api_calls_made += 1
                
            except requests.exceptions.Timeout:
                raise TimeoutError("Timeout: L'API n'a pas répondu dans les 30 secondes")
            except requests.exceptions.ConnectionError:
                raise ConnectionError("Erreur de connexion: Impossible de joindre l'API")
            except requests.exceptions.RequestException as e:
                raise RuntimeError(f"Erreur requête HTTP: {str(e)}")
            
            # Étape 3: Vérifier le statut HTTP
            if response.status_code == 429:
                raise RuntimeError("Rate limit atteint (429). Augmentez delay_between_calls")
            elif response.status_code == 401:
                raise RuntimeError("Authentification échouée (401). Vérifiez votre token API")
            elif response.status_code == 503:
                raise RuntimeError("Service indisponible (503). Réessayez plus tard")
            elif not response.ok:
                raise RuntimeError(f"Erreur HTTP {response.status_code}: {response.text[:100]}")
            
            # Étape 4: Parser la réponse JSON
            try:
                results = response.json()
            except ValueError:
                raise ValueError("Réponse API invalide (non-JSON)")
            
            # Étape 5: Créer le masque combiné
            width, height = get_image_dimensions(img_path)
            segmentation_mask = create_masks(results, width, height)
            batch_seg_results.append(segmentation_mask)
            print(f"    Succès: {width}x{height} pixels\n")

        except FileNotFoundError as e:
            print(f"    Erreur fichier: {e}\n")
            batch_seg_results.append(None)
        except TimeoutError as e:
            print(f"    Erreur timeout: {e}\n")
            batch_seg_results.append(None)
        except ConnectionError as e:
            print(f"    Erreur connexion: {e}\n")
            batch_seg_results.append(None)
        except ValueError as e:
            print(f"    Erreur parsing: {e}\n")
            batch_seg_results.append(None)
        except RuntimeError as e:
            print(f"    Erreur API: {e}\n")
            batch_seg_results.append(None)
        except Exception as e:
            print(f"    Erreur inattendue: {str(e)[:100]}\n")
            batch_seg_results.append(None)

        # Délai entre les appels
        time.sleep(delay_between_calls)

    # Résumé du traitement
    print("="*60)
    print(f"RÉSUMÉ DU TRAITEMENT:")
    print(f"  Images traitées: {api_calls_made}")
    print(f"  Appels API effectués: {api_calls_made}/{max_api_calls}")
    print(f"  Images ignorées (limite): {images_skipped}")
    print(f"  Images en erreur: {sum(1 for r in batch_seg_results if r is None)}")
    print("="*60 + "\n")

    return batch_seg_results


## 6. Affichage des Résultats en Batch



In [ ]:
def display_segmented_images_batch(original_image_paths, segmentation_masks):
    """
    Affiche: Image originale | Masque colorisé + Légende | Superposition + Légende.
    Légende intégrée dans les images sans bordure, en transparence 100% et texte blanc.
    """
    
    if not original_image_paths or not segmentation_masks:
        print("Listes vides.")
        return
    
    valid_pairs = []
    
    for img_path, mask_array in zip(original_image_paths, segmentation_masks):
        if mask_array is None or not os.path.exists(img_path):
            continue
        
        original_img = Image.open(img_path)
        if original_img.mode != 'RGB':
            original_img = original_img.convert('RGB')
        
        valid_pairs.append((img_path, original_img, mask_array))
    
    if not valid_pairs:
        print("Aucune paire valide.")
        return
    
    valid_count = len(valid_pairs)
    fig = plt.figure(figsize=(20, valid_count * 6))
    
    # Créer la légende une fois (sans "Arrière-plan")
    legend_elements = []
    for class_name_en in sorted(CLASS_MAPPING.keys()):
        # Filtrer "Background" (Arrière-plan)
        if class_name_en == "Background":
            continue
        
        class_id = CLASS_MAPPING[class_name_en]
        class_name_fr = CLASS_NAMES_FRENCH.get(class_name_en, class_name_en)
        color_rgb = tuple(c / 255.0 for c in COLOR_MAPPING[class_id])
        legend_elements.append(
            mpatches.Patch(facecolor=color_rgb, edgecolor='black', linewidth=0.8,
                          label=class_name_fr)
        )
    
    for row_idx, (img_path, original_img, mask_array) in enumerate(valid_pairs):
        
        # Colonne 1: Image originale
        ax1 = plt.subplot(valid_count, 3, row_idx * 3 + 1)
        ax1.imshow(original_img)
        ax1.set_title("Image originale", fontsize=12, fontweight='bold')
        ax1.axis('off')
        
        # Colonne 2: Masque colorisé + Légende intégrée
        colored_mask = colorize_segmentation_mask(mask_array)
        ax2 = plt.subplot(valid_count, 3, row_idx * 3 + 2)
        ax2.imshow(colored_mask)
        ax2.set_title("Masque colorisé", fontsize=12, fontweight='bold')
        ax2.axis('off')
        
        # Légende intégrée sans bordure en transparence 100%
        legend2 = ax2.legend(handles=legend_elements, loc='upper left', fontsize=8, 
                            frameon=False,  # Pas de bordure
                            framealpha=1.0,  # Transparence 100%
                            bbox_to_anchor=(0.02, 0.98))
        
        # Changer la couleur du texte en blanc
        for text in legend2.get_texts():
            text.set_color('white')
        
        # Colonne 3: Superposition + Légende intégrée
        overlay_img = create_overlay(original_img, colored_mask, alpha=0.5)
        ax3 = plt.subplot(valid_count, 3, row_idx * 3 + 3)
        ax3.imshow(overlay_img)
        ax3.set_title("Superposition", fontsize=12, fontweight='bold')
        ax3.axis('off')
        
        # Légende intégrée sans bordure en transparence 100%
        legend3 = ax3.legend(handles=legend_elements, loc='upper left', fontsize=8, 
                            frameon=False,  # Pas de bordure
                            framealpha=1.0,  # Transparence 100%
                            bbox_to_anchor=(0.02, 0.98))
        
        # Changer la couleur du texte en blanc
        for text in legend3.get_texts():
            text.set_color('white')
    
    plt.tight_layout()
    plt.show()

# ============================================================================
# EXÉCUTION
# ============================================================================

if image_paths:
    print(f"Traitement de {len(image_paths)} image(s)...\n")
    batch_seg_results = segment_images_batch(
        image_paths,
        max_api_calls=50,      # Limite à 50 appels API
        delay_between_calls=1   # 1 seconde entre chaque appel (ou 3 si vous préférez)
    )
    print("Affichage des résultats...\n")
    display_segmented_images_batch(image_paths, batch_seg_results)
else:
    print("Aucune image trouvée.")

# 7. Evaluation du modèle

In [ ]:
pip install sklearn

In [ ]:
# ============================================================================
# SYSTÈME D'ÉVALUATION DU MODÈLE - SEGFORMER B3 CLOTHES
# ============================================================================

"""
Évaluation du Modèle: sayeed99/segformer_b3_clothes
Source: https://huggingface.co/sayeed99/segformer_b3_clothes

Métriques Officielles HF:
  - Mean IoU: 0.69
  - Mean Accuracy: 0.80
  - Evaluation Loss: 0.15
"""

import numpy as np
from sklearn.metrics import jaccard_score, accuracy_score, f1_score
import pandas as pd


class ModelEvaluator:
    """
    Classe pour évaluer les performances du modèle SegFormer B3.
    Compare les résultats avec les benchmarks officiels HF.
    """
    
    # Métriques officielles du modèle depuis HuggingFace
    OFFICIAL_METRICS_BY_CLASS = {
        0: {"name": "Background", "accuracy": 0.99, "iou": 0.99},
        1: {"name": "Hat", "accuracy": 0.73, "iou": 0.68},
        2: {"name": "Hair", "accuracy": 0.91, "iou": 0.82},
        3: {"name": "Sunglasses", "accuracy": 0.73, "iou": 0.63},
        4: {"name": "Upper-clothes", "accuracy": 0.87, "iou": 0.78},
        5: {"name": "Skirt", "accuracy": 0.76, "iou": 0.65},
        6: {"name": "Pants", "accuracy": 0.90, "iou": 0.84},
        7: {"name": "Dress", "accuracy": 0.74, "iou": 0.55},
        8: {"name": "Belt", "accuracy": 0.35, "iou": 0.30},
        9: {"name": "Left-shoe", "accuracy": 0.74, "iou": 0.58},
        10: {"name": "Right-shoe", "accuracy": 0.75, "iou": 0.60},
        11: {"name": "Face", "accuracy": 0.92, "iou": 0.85},
        12: {"name": "Left-leg", "accuracy": 0.90, "iou": 0.82},
        13: {"name": "Right-leg", "accuracy": 0.90, "iou": 0.81},
        14: {"name": "Left-arm", "accuracy": 0.86, "iou": 0.74},
        15: {"name": "Right-arm", "accuracy": 0.82, "iou": 0.73},
        16: {"name": "Bag", "accuracy": 0.91, "iou": 0.84},
        17: {"name": "Scarf", "accuracy": 0.63, "iou": 0.29},
    }
    
    OFFICIAL_OVERALL_METRICS = {
        "mean_accuracy": 0.80,
        "mean_iou": 0.69,
        "evaluation_loss": 0.15,
    }
    
    def __init__(self):
        """Initialise l'évaluateur du modèle."""
        self.evaluation_results = {
            "by_class": {},
            "overall": {},
        }
    
    # =========================================================================
    # CALCUL DES MÉTRIQUES
    # =========================================================================
    
    @staticmethod
    def calculate_iou(mask_pred, mask_true):
        """
        Calcule l'IoU (Intersection over Union) pour une classe.
        
        IoU = |Intersection| / |Union|
        Metrique standard pour la segmentation d'images.
        
        Args:
            mask_pred (np.ndarray): Masque prédiction binaire
            mask_true (np.ndarray): Masque vérité terrain binaire
            
        Returns:
            float: Score IoU entre 0 et 1
        """
        try:
            iou = jaccard_score(mask_true.flatten(), mask_pred.flatten(), 
                               zero_division=0)
            return iou
        except Exception:
            return 0.0
    
    @staticmethod
    def calculate_accuracy(mask_pred, mask_true):
        """
        Calcule l'Accuracy (pixels correctement classifiés).
        
        Accuracy = |Pixels corrects| / |Total pixels|
        
        Args:
            mask_pred (np.ndarray): Masque prédiction binaire
            mask_true (np.ndarray): Masque vérité terrain binaire
            
        Returns:
            float: Score Accuracy entre 0 et 1
        """
        try:
            acc = accuracy_score(mask_true.flatten(), mask_pred.flatten())
            return acc
        except Exception:
            return 0.0
    
    @staticmethod
    def calculate_f1_score(mask_pred, mask_true):
        """
        Calcule le F1 Score (harmonic mean).
        
        F1 = 2 * (Precision * Recall) / (Precision + Recall)
        
        Args:
            mask_pred (np.ndarray): Masque prédiction binaire
            mask_true (np.ndarray): Masque vérité terrain binaire
            
        Returns:
            float: Score F1 entre 0 et 1
        """
        try:
            f1 = f1_score(mask_true.flatten(), mask_pred.flatten(), 
                         zero_division=0)
            return f1
        except Exception:
            return 0.0
    
    # =========================================================================
    # ÉVALUATION GLOBALE
    # =========================================================================
    
    def display_official_metrics(self):
        """Affiche les métriques officielles du modèle HF."""
        
        print("\n" + "="*80)
        print("METRIQUES OFFICIELLES - MODELE HUGGING FACE")
        print("="*80)
        
        print(f"\nMODEL CARD:")
        print(f"  Model ID:                  sayeed99/segformer_b3_clothes")
        print(f"  Architecture:              SegFormer B3")
        print(f"  Task:                      Image Segmentation")
        print(f"  Dataset:                   mattmdjaga/human_parsing_dataset")
        print(f"  License:                   MIT")
        print(f"  Downloads (last month):    8,051")
        
        print(f"\nMETRIQUES GLOBALES:")
        print(f"  Mean Accuracy:             {self.OFFICIAL_OVERALL_METRICS['mean_accuracy']:.2%}")
        print(f"  Mean IoU:                  {self.OFFICIAL_OVERALL_METRICS['mean_iou']:.2%}")
        print(f"  Evaluation Loss:           {self.OFFICIAL_OVERALL_METRICS['evaluation_loss']:.4f}")
        
        print(f"\nPERFORMANCES PAR CLASSE (Top 5 - Meilleur IoU):")
        sorted_classes = sorted(self.OFFICIAL_METRICS_BY_CLASS.items(),
                               key=lambda x: x[1]["iou"],
                               reverse=True)[:5]
        for class_id, metrics in sorted_classes:
            print(f"  {metrics['name']:15} (ID={class_id:2}) | Accuracy: {metrics['accuracy']:.2%} | IoU: {metrics['iou']:.2%}")
        
        print(f"\nPERFORMANCES PAR CLASSE (Bottom 5 - Pire IoU):")
        worst_classes = sorted(self.OFFICIAL_METRICS_BY_CLASS.items(),
                              key=lambda x: x[1]["iou"])[:5]
        for class_id, metrics in worst_classes:
            print(f"  {metrics['name']:15} (ID={class_id:2}) | Accuracy: {metrics['accuracy']:.2%} | IoU: {metrics['iou']:.2%}")
        
        print("="*80)
    
    def display_evaluation_table(self):
        """Affiche le tableau complet des évaluations par classe."""
        
        print("\n" + "="*80)
        print("TABLEAU D'EVALUATION PAR CLASSE")
        print("="*80 + "\n")
        
        # Créer dataframe
        data = []
        for class_id in sorted(self.OFFICIAL_METRICS_BY_CLASS.keys()):
            metrics = self.OFFICIAL_METRICS_BY_CLASS[class_id]
            data.append({
                "Label ID": class_id,
                "Classe": metrics["name"],
                "Accuracy": f"{metrics['accuracy']:.2%}",
                "IoU": f"{metrics['iou']:.2%}",
                "Performance": "Excellent" if metrics["iou"] >= 0.80 
                              else "Bon" if metrics["iou"] >= 0.60
                              else "Moyen" if metrics["iou"] >= 0.30
                              else "Faible"
            })
        
        df = pd.DataFrame(data)
        print(df.to_string(index=False))
        
        print("\n" + "="*80)
    
    # =========================================================================
    # ANALYSE DE PERFORMANCE
    # =========================================================================
    
    def display_performance_analysis(self):
        """Affiche une analyse détaillée des performances."""
        
        print("\n" + "="*80)
        print("ANALYSE DETAILLEE DE PERFORMANCE")
        print("="*80)
        
        # Grouper par catégories de performance
        excellent = []
        good = []
        average = []
        weak = []
        
        for class_id, metrics in self.OFFICIAL_METRICS_BY_CLASS.items():
            iou = metrics["iou"]
            if iou >= 0.80:
                excellent.append((metrics["name"], iou))
            elif iou >= 0.60:
                good.append((metrics["name"], iou))
            elif iou >= 0.30:
                average.append((metrics["name"], iou))
            else:
                weak.append((metrics["name"], iou))
        
        print(f"\nPERFORMANCE EXCELLENTE (IoU >= 0.80):")
        print(f"  Nombre de classes: {len(excellent)}/18")
        for name, iou in sorted(excellent, key=lambda x: x[1], reverse=True):
            print(f"    {name:20} IoU: {iou:.2%}")
        
        print(f"\nBONNE PERFORMANCE (0.60 <= IoU < 0.80):")
        print(f"  Nombre de classes: {len(good)}/18")
        for name, iou in sorted(good, key=lambda x: x[1], reverse=True):
            print(f"    {name:20} IoU: {iou:.2%}")
        
        print(f"\nPERFORMANCE MOYENNE (0.30 <= IoU < 0.60):")
        print(f"  Nombre de classes: {len(average)}/18")
        for name, iou in sorted(average, key=lambda x: x[1], reverse=True):
            print(f"    {name:20} IoU: {iou:.2%}")
        
        print(f"\nPERFORMANCE FAIBLE (IoU < 0.30):")
        print(f"  Nombre de classes: {len(weak)}/18")
        for name, iou in sorted(weak, key=lambda x: x[1], reverse=True):
            print(f"    {name:20} IoU: {iou:.2%}")
        
        print("="*80)
    
    # =========================================================================
    # MATRICE DE CONFUSION CONCEPTUELLE
    # =========================================================================
    
    def display_confusion_analysis(self):
        """Affiche une analyse des classes confondues."""
        
        print("\n" + "="*80)
        print("ANALYSE DES CONFUSIONS PROBABLES")
        print("="*80)
        
        print(f"\nCLASSES FREQUEMMENT CONFONDUES (faible IoU):")
        print(f"\n1. Ceinture (Belt) - IoU: 0.30")
        print(f"   Raisons possibles:")
        print(f"     - Classe tres petite (peu de pixels)")
        print(f"     - Souvent confondue avec le haut (Upper-clothes)")
        print(f"     - Variation visuelle importante")
        
        print(f"\n2. Echarpe (Scarf) - IoU: 0.29")
        print(f"   Raisons possibles:")
        print(f"     - Classe minoritaire dans le dataset")
        print(f"     - Segmentation difficile (formes variables)")
        print(f"     - Souvent confondue avec les cheveux (Hair)")
        
        print(f"\n3. Robe (Dress) - IoU: 0.55")
        print(f"   Raisons possibles:")
        print(f"     - Classe ambigue (peut ressembler à un haut + jupe)")
        print(f"     - Variations de longueur importantes")
        
        print(f"\nCLASSES BIEN SEGMENTEES (bon IoU):")
        print(f"\n1. Fond (Background) - IoU: 0.99")
        print(f"   Force: Classe majoritaire, bien definie")
        
        print(f"\n2. Visage (Face) - IoU: 0.85")
        print(f"   Force: Texture distincte, bien identifiable")
        
        print(f"\n3. Pantalon (Pants) - IoU: 0.84")
        print(f"   Force: Classe courante, distincte")
        
        print("="*80)
    
    # =========================================================================
    # RECOMMANDATIONS D'AMÉLIORATION
    # =========================================================================
    
    def display_improvement_recommendations(self):
        """Affiche les recommandations pour améliorer le modèle."""
        
        print("\n" + "="*80)
        print("RECOMMANDATIONS D'AMELIORATION")
        print("="*80)
        
        print(f"\nPOUR LES CLASSES FAIBLES (IoU < 0.60):")
        
        print(f"\n1. Ceinture (Belt) - Priorite HAUTE:")
        print(f"   Actions recommandees:")
        print(f"     - Augmenter les donnees d'entrainement pour cette classe")
        print(f"     - Utiliser data augmentation specifique")
        print(f"     - Revoir les labels (peut-etre confondus)")
        print(f"     - Ajuster les poids de classe (class weighting)")
        
        print(f"\n2. Echarpe (Scarf) - Priorite HAUTE:")
        print(f"   Actions recommandees:")
        print(f"     - Collecter plus d'exemples d'echarpes")
        print(f"     - Augmenter le poids de cette classe")
        print(f"     - Envisager une fine-tuning specifique")
        
        print(f"\n3. Robe (Dress) - Priorite MOYENNE:")
        print(f"   Actions recommandees:")
        print(f"     - Post-processing: regles heuristiques")
        print(f"     - Combiner avec detection de motifs")
        
        print(f"\nSTRATEGIES GENERALES D'AMELIORATION:")
        print(f"  - Augmenter la resolution des images d'entrainement")
        print(f"  - Implémenter un post-processing morphologique")
        print(f"  - Utiliser un ensemble (ensemble learning)")
        print(f"  - Faire une fine-tuning sur des donnees specifiques")
        print(f"  - Augmenter la capacite du modele (B4 ou B5)")
        
        print("="*80)
    
    # =========================================================================
    # RAPPORT SYNTHÉTIQUE
    # =========================================================================
    
    def display_complete_evaluation_report(self):
        """Affiche le rapport d'évaluation complet."""
        
        print("\n\n")
        print("="*80)
        print("RAPPORT D'EVALUATION COMPLET DU MODELE")
        print("SegFormer B3 Clothes - sayeed99/segformer_b3_clothes")
        print("="*80)
        
        # Afficher toutes les sections
        self.display_official_metrics()
        self.display_evaluation_table()
        self.display_performance_analysis()
        self.display_confusion_analysis()
        self.display_improvement_recommendations()
        
        # Résumé final
        print("\n" + "="*80)
        print("RESUME EXECUTIF")
        print("="*80)
        
        print(f"\nEtat global du modele:")
        print(f"  Accuracy moyenne:          80.0%")
        print(f"  IoU moyenne:               69.0%")
        print(f"  Nombre de classes:         18")
        print(f"  Classes excellentes:       7/18 (39%)")
        print(f"  Classes en difficulte:     4/18 (22%)")
        
        print(f"\nConclude:")
        print(f"  Le modele SegFormer B3 presente une performance solide")
        print(f"  pour la majorite des vetements. Les axes d'amelioration")
        print(f"  prioritaires sont les accessoires (ceinture, echarpe)")
        print(f"  et certains vetements ambigus (robe).")
        print(f"\n  Utilisation recommandee: Production avec care sur")
        print(f"  les classes faibles. Fine-tuning recommande pour")
        print(f"  applications specifiques.")
        
        print("\n" + "="*80 + "\n")


# ============================================================================
# EXECUTION
# ============================================================================

if __name__ == "__main__":
    
    # Creer l'évaluateur
    evaluator = ModelEvaluator()
    
    # Afficher le rapport complet d'évaluation
    evaluator.display_complete_evaluation_report()

# 8. Estimation des couts

In [ ]:
# ============================================================================
# ESTIMATION DE CHIFFRAGE FORMALISÉE - 1 IMAGE vs 50 EN BATCH
# ============================================================================

"""
Estimation de Chiffrage du Projet de Segmentation de Vêtements
Modèle: sayeed99/segformer_b3_clothes
Comparaison: 1 Image Seule vs 50 Images en Batch
"""

from datetime import datetime, timedelta
import pandas as pd


class EstimationChiffrage:
    """
    Classe pour formaliser l'estimation de chiffrage en 3 dimensions.
    Basée sur les tarifs HF officiels et les métriques du modèle.
    """
    
    def __init__(self, num_images=50):
        """
        Initialise l'estimation.
        
        Args:
            num_images (int): Nombre d'images à traiter
        """
        self.num_images = num_images
        self.model_id = "sayeed99/segformer_b3_clothes"
        self.date_estimation = datetime.now()
        
        # Métriques officielles du modèle
        self.model_metrics = {
            "mean_iou": 0.69,
            "mean_accuracy": 0.80,
            "evaluation_loss": 0.15,
        }
        
        # Tarifs Hugging Face
        self.hf_pricing = {
            "gpu_compute_per_second": 0.00008,
            "storage_per_gb_month": 0.023,
            "bandwidth_per_gb": 0.12,
        }
        
        # Parametres temporels
        self.timing_params = {
            "api_call_per_image_seconds": 2.5,
            "network_overhead_per_image": 1.0,
            "rate_limit_delay_single": 0.5,  # Pas de delai important pour 1 image
            "rate_limit_delay_batch": 1.0,   # Delai entre chaque appel en batch
        }
    
    def calculate_timing_and_costs(self, scenario="batch"):
        """
        Calcule temps et coûts selon le scénario.
        
        Args:
            scenario (str): "single" pour 1 image, "batch" pour batch
            
        Returns:
            dict: Données temporelles et financières
        """
        if scenario == "single":
            rate_limit_delay = self.timing_params["rate_limit_delay_single"]
        else:
            rate_limit_delay = self.timing_params["rate_limit_delay_batch"]
        
        # Temps par appel
        time_per_call = (self.timing_params["api_call_per_image_seconds"] +
                        self.timing_params["network_overhead_per_image"] +
                        rate_limit_delay)
        
        total_inference_seconds = self.num_images * time_per_call
        
        # Coûts
        cost_api = total_inference_seconds * self.hf_pricing["gpu_compute_per_second"]
        storage_gb = (self.num_images * 2) / 1024
        cost_storage = storage_gb * self.hf_pricing["storage_per_gb_month"]
        bandwidth_gb = (self.num_images * 4) / 1024
        cost_bandwidth = bandwidth_gb * self.hf_pricing["bandwidth_per_gb"]
        total_cost = cost_api + cost_storage + cost_bandwidth
        
        return {
            "images": self.num_images,
            "time_per_call_seconds": time_per_call,
            "total_seconds": total_inference_seconds,
            "total_minutes": total_inference_seconds / 60,
            "total_hours": total_inference_seconds / 3600,
            "cost_api": cost_api,
            "cost_storage": cost_storage,
            "cost_bandwidth": cost_bandwidth,
            "total_cost": total_cost,
            "cost_per_image": total_cost / self.num_images,
        }


# ============================================================================
# ESTIMATION 1 IMAGE SEULE
# ============================================================================

print("\n" + "="*80)
print("1. ESTIMATION CHIFFRAGE - 1 IMAGE SEULE")
print("="*80)

estimation_1 = EstimationChiffrage(num_images=1)
data_1 = estimation_1.calculate_timing_and_costs(scenario="single")

print(f"\nTEMPS:")
print(f"  Secondes:                   {data_1['total_seconds']:.1f} sec")
print(f"  Minutes:                    {data_1['total_minutes']:.2f} min")

print(f"\nCOUTS (Tarifs HF):")
print(f"  API Inference:              ${data_1['cost_api']:.6f}")
print(f"  Stockage:                   ${data_1['cost_storage']:.6f}")
print(f"  Bandwidth:                  ${data_1['cost_bandwidth']:.6f}")
print(f"  Total:                      ${data_1['total_cost']:.6f}")

print(f"\nCREDITS GRATUITS:")
print(f"  Free User (0.10/mois):      COUVERT (cout < 0.10)")
print(f"  Pro User (2.00/mois):       COUVERT (cout < 2.00)")

print("="*80)


# ============================================================================
# ESTIMATION 50 IMAGES EN BATCH
# ============================================================================

print("\n" + "="*80)
print("2. ESTIMATION CHIFFRAGE - 50 IMAGES EN BATCH")
print("="*80)

estimation_50 = EstimationChiffrage(num_images=50)
data_50 = estimation_50.calculate_timing_and_costs(scenario="batch")

print(f"\nTEMPS:")
print(f"  Secondes:                   {data_50['total_seconds']:.1f} sec")
print(f"  Minutes:                    {data_50['total_minutes']:.2f} min")
print(f"  Heures:                     {data_50['total_hours']:.3f} h")

print(f"\nCOUTS (Tarifs HF):")
print(f"  API Inference:              ${data_50['cost_api']:.6f}")
print(f"  Stockage:                   ${data_50['cost_storage']:.6f}")
print(f"  Bandwidth:                  ${data_50['cost_bandwidth']:.6f}")
print(f"  Total:                      ${data_50['total_cost']:.6f}")

print(f"\nCREDITS GRATUITS:")
print(f"  Free User (0.10/mois):      COUVERT (cout < 0.10)")
print(f"  Pro User (2.00/mois):       COUVERT (cout < 2.00)")

print("="*80)


# ============================================================================
# SCENARIO COMPARATIF: 50 Images en 1 par 1 vs 50 en Batch
# ============================================================================

print("\n" + "="*80)
print("3. COMPARAISON SCENARIO - 50 IMAGES: 1 PAR 1 vs EN BATCH")
print("="*80)

# Scenario 1: 50 images traitées 1 par 1
estimation_50_solo = EstimationChiffrage(num_images=50)
data_50_solo = estimation_50_solo.calculate_timing_and_costs(scenario="single")

# Scenario 2: 50 images en batch
estimation_50_batch = EstimationChiffrage(num_images=50)
data_50_batch = estimation_50_batch.calculate_timing_and_costs(scenario="batch")

# Créer tableau comparatif
comparison_data = {
    "Metrique": [
        "Nombre d'images",
        "Temps par appel (sec)",
        "Temps total (sec)",
        "Temps total (min)",
        "Temps total (heures)",
        "",
        "Coût API (USD)",
        "Coût Stockage (USD)",
        "Coût Bandwidth (USD)",
        "Coût Total (USD)",
        "Coût par image (USD)",
        "",
        "Gain de temps (%)",
        "Economies (USD)",
        "Economies (%)",
    ],
    "50 Images 1 par 1": [
        str(data_50_solo["images"]),
        f"{data_50_solo['time_per_call_seconds']:.1f}",
        f"{data_50_solo['total_seconds']:.1f}",
        f"{data_50_solo['total_minutes']:.1f}",
        f"{data_50_solo['total_hours']:.3f}",
        "",
        f"${data_50_solo['cost_api']:.6f}",
        f"${data_50_solo['cost_storage']:.6f}",
        f"${data_50_solo['cost_bandwidth']:.6f}",
        f"${data_50_solo['total_cost']:.6f}",
        f"${data_50_solo['cost_per_image']:.6f}",
        "",
        "-",
        "-",
        "-",
    ],
    "50 Images Batch": [
        str(data_50_batch["images"]),
        f"{data_50_batch['time_per_call_seconds']:.1f}",
        f"{data_50_batch['total_seconds']:.1f}",
        f"{data_50_batch['total_minutes']:.1f}",
        f"{data_50_batch['total_hours']:.3f}",
        "",
        f"${data_50_batch['cost_api']:.6f}",
        f"${data_50_batch['cost_storage']:.6f}",
        f"${data_50_batch['cost_bandwidth']:.6f}",
        f"${data_50_batch['total_cost']:.6f}",
        f"${data_50_batch['cost_per_image']:.6f}",
        "",
        f"{((data_50_solo['total_seconds']-data_50_batch['total_seconds'])/data_50_solo['total_seconds']*100):.1f}%",
        f"${data_50_solo['total_cost']-data_50_batch['total_cost']:.6f}",
        f"{((data_50_solo['total_cost']-data_50_batch['total_cost'])/data_50_solo['total_cost']*100):.1f}%",
    ]
}

# Afficher le tableau
df = pd.DataFrame(comparison_data)
print("\n")
print(df.to_string(index=False))

# Analyse
print("\n" + "="*80)
print("ANALYSE COMPARATIVE")
print("="*80)

gain_temps = ((data_50_solo['total_seconds']-data_50_batch['total_seconds'])/data_50_solo['total_seconds']*100)
economies = data_50_solo['total_cost']-data_50_batch['total_cost']
economies_pct = (economies/data_50_solo['total_cost']*100)

print(f"\nGAIN DE TEMPS:")
print(f"  1 par 1:                    {data_50_solo['total_minutes']:.1f} minutes")
print(f"  En batch:                   {data_50_batch['total_minutes']:.1f} minutes")
print(f"  Gain:                       {data_50_solo['total_minutes']-data_50_batch['total_minutes']:.1f} minutes")
print(f"  Gain en pourcentage:        {gain_temps:.1f}%")

print(f"\nECONOMIES FINANCIERES:")
print(f"  Cout 1 par 1:               ${data_50_solo['total_cost']:.6f}")
print(f"  Cout en batch:              ${data_50_batch['total_cost']:.6f}")
print(f"  Economies:                  ${economies:.6f}")
print(f"  Economies en pourcentage:   {economies_pct:.1f}%")

print(f"\nCONCLUSION:")
if gain_temps > 0:
    print(f"  Traiter 50 images en batch est plus rapide de {gain_temps:.1f}%")
    print(f"  et moins cher de {economies_pct:.1f}%")
else:
    print(f"  Les deux approches ont des performances similaires")

print("="*80)


# ============================================================================
# TABLEAU SYNTHÉTIQUE FINAL
# ============================================================================

print("\n" + "="*80)
print("TABLEAU SYNTHÉTIQUE - RÉSUMÉ GLOBAL")
print("="*80)

summary_data = {
    "Scenario": [
        "1 Image seule",
        "50 Images 1 par 1",
        "50 Images en Batch",
    ],
    "Temps (sec)": [
        f"{data_1['total_seconds']:.1f}",
        f"{data_50_solo['total_seconds']:.1f}",
        f"{data_50_batch['total_seconds']:.1f}",
    ],
    "Temps (min)": [
        f"{data_1['total_minutes']:.2f}",
        f"{data_50_solo['total_minutes']:.1f}",
        f"{data_50_batch['total_minutes']:.1f}",
    ],
    "Cout (USD)": [
        f"${data_1['total_cost']:.6f}",
        f"${data_50_solo['total_cost']:.6f}",
        f"${data_50_batch['total_cost']:.6f}",
    ],
    "Cout/Image (USD)": [
        f"${data_1['cost_per_image']:.6f}",
        f"${data_50_solo['cost_per_image']:.6f}",
        f"${data_50_batch['cost_per_image']:.6f}",
    ],
    "Efficacite": [
        "Ref",
        "Moins efficace",
        "Plus efficace",
    ]
}

df_summary = pd.DataFrame(summary_data)
print("\n")
print(df_summary.to_string(index=False))

print("\n" + "="*80 + "\n")

# - FIN - 